###  1. csv를 읽어서 RDD를 생성하고, 5줄을 화면출력하세요.

결과가 깨져보인다면, 그 이유를 간단히 적으세요.

* 1) 경기도 의정부시 인구현황 (파일명: ```경기도 의정부시_인구현황_20200904```)

https://www.data.go.kr/data/15009613/fileData.do

In [2]:
import pyspark
import sys
import os

os.environ["PYSPARK_PYTHON"]=sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"]=sys.executable

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf=myConf)\
        .getOrCreate()


In [9]:
path = os.path.join('data','경기도 의정부시_인구현황_20210910.csv')
rdd = spark.sparkContext.textFile(path, use_unicode=True)
rdd.take(2)

['�������,�α���(��),�α���(��),�α���(��),������(��),������(��),������(��),����,�����,������α�,���������,�����μ���,�μ���ȭ��ȣ,�����ͱ�������',
 '������1��,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,�����ν�,�ο����ǰ�,031-828-2466,2021-09-10']

In [13]:
newrdd = spark.sparkContext.binaryFiles(path)
_my = newrdd.map(lambda x :x[1].decode('euc-kr'))
_my.take(2)

[('file:/Users/nadonghyeon/BigData/201721402/data/경기도 의정부시_인구현황_20210910.csv',
  b'\xc7\xe0\xc1\xa4\xb1\xe2\xb0\xfc,\xc0\xce\xb1\xb8\xbc\xf6(\xb0\xe8),\xc0\xce\xb1\xb8\xbc\xf6(\xb3\xb2),\xc0\xce\xb1\xb8\xbc\xf6(\xbf\xa9),\xb1\xb8\xbc\xba\xba\xf1(\xb0\xe8),\xb1\xb8\xbc\xba\xba\xf1(\xb3\xb2),\xb1\xb8\xbc\xba\xba\xf1(\xbf\xa9),\xbc\xba\xba\xf1,\xbc\xbc\xb4\xeb\xbc\xf6,\xbc\xbc\xb4\xeb\xb4\xe7\xc0\xce\xb1\xb8,\xb0\xfc\xb8\xae\xb1\xe2\xb0\xfc\xb8\xed,\xb0\xfc\xb8\xae\xba\xce\xbc\xad\xb8\xed,\xba\xce\xbc\xad\xc0\xfc\xc8\xad\xb9\xf8\xc8\xa3,\xb5\xa5\xc0\xcc\xc5\xcd\xb1\xe2\xc1\xd8\xc0\xcf\xc0\xda\r\n\xc0\xc7\xc1\xa4\xba\xce1\xb5\xbf,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,\xc0\xc7\xc1\xa4\xba\xce\xbd\xc3,\xb9\xce\xbf\xf8\xbf\xa9\xb1\xc7\xb0\xfa,031-828-2466,2021-09-10\r\n\xc0\xc7\xc1\xa4\xba\xce2\xb5\xbf,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,\xc0\xc7\xc1\xa4\xba\xce\xbd\xc3,\xb9\xce\xbf\xf8\xbf\xa9\xb1\xc7\xb0\xfa,031-828-2466,2021-09-10\r\n\xc8\xa3\xbf\xf81\xb5\xbf,36124

* 2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 (파일명: ```제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623```)

In [15]:
path = os.path.join('data','제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv')
rdd = spark.sparkContext.binaryFiles(path)
_my = rdd.map(lambda x :x[1].decode('euc-kr'))
_my.collect()

['연도별,서귀포시 인구수,65세이상 인구수 ,14세이하 인구수,고령화비율,노령화지수,데이터기준일자\r\n2012,154057,25826,22861,16.76,112.97,2021-08-31\r\n2013,155641,26936,22393,17.31,120.29,2021-08-31\r\n2014,158512,27877,22058,17.59,126.38,2021-08-31\r\n2015,164519,28979,22362,17.61,129.59,2021-08-31\r\n2016,170932,30030,23044,17.57,130.32,2021-08-31\r\n2017,173302,30545,23300,17.63,131.09,2021-08-31\r\n2018,181245,32394,24013,17.87,134.9,2021-08-31\r\n2019,181584,33707,23707,18.56,142.18,2021-08-31\r\n2020,181059,34456,23579,19.03,146.13,2021-08-31\r\n2021,183015,36353,23759,19.86,141.12,2021-08-31\r\n']

### 2. 텍스트 파일을 읽고 RDD를 사용하여 단어빈도를 계산하고, 그래프 그리기.

In [50]:
path = os.path.join('data','ds_bigdata_wiki.txt')
rdd = spark.sparkContext.textFile(path)
stopwords = set(['및','이를','등','이','이런','그와','또는','두', '이와', '전', '간'])

newlist = rdd.flatMap(lambda x : x.split())\
        .filter(lambda x : x.lower() not in stopwords)\
        .map(lambda x : (x,1))\
        .reduceByKey(lambda x,y: x+y)\
        .map(lambda x : (x[1],x[0]))\
        .sortByKey(False)\
        .collect()
for k,v in newlist:
    print(k,  v)


21 데이터
18 데이터를
14 빅
9 있다.
8 수
8 데이터의
7 미국
7 통해
6 유권자
6 선거
6 대한
6 빅데이터
5 활용한
5 소셜
5 대한민국
5 위한
5 있는
4 오바마
4 맞춤형
4 같은
4 SNS
4 구글
4 비정형
4 리그
4 방대한
4 독일
4 분석
4 의료
4 자동
4 번역
3 대통령
3 분석,
3 데이터는
3 비용
3 제19대
3 네트워크
3 방식으로
3 분석해
3 활용을
3 머니볼
3 빌리
3 최근
3 더욱
3 선수의
3 아니라
3 프로그램
3 매치
3 결과를
3 약
3 이러한
3 등의
3 있어
3 것으로
3 빅데이터를
3 효과를
3 스토리지
3 IBM의
3 시스템
3 건의
2 2008년
2 다양한
2 데이터베이스를
2 캠프는
2 것을
2 투표
2 정보를
2 효과적인
2 있었다.
2 이에
2 2010년
2 제5회
2 지방
2 2011년
2 활용에
2 가운데
2 여론
2 기존
2 여론조사
2 큰
2 기술을
2 분석을
2 그러나
2 이용자의
2 분석은
2 경영[편집]
2 아마존닷컴의
2 추천
2 상품
2 모든
2 고객
2 한사람
2 제시하는
2 나아가
2 제공하는
2 기반으로
2 메이저
2 해당하는
2 하는
2 과학기술
2 수집이
2 이처럼
2 정형
2 커지고
2 성적을
2 나타내는
2 야구를
2 베이스를
2 밟은
2 총
2 한
2 숫자로
2 생활
2 활용되는
2 '빅데이터'
2 기법을
2 분석한
2 2014년
2 FIFA
2 끝에
2 국가대표팀은
2 SAP
2 센서를
2 수집,
2 기존에
2 더
2 정보
2 선수당
2 포함해
2 대량의
2 관한
2 데이터과학자
2 보유하고
2 한편,
2 활용하면
2 의사결정을
2 가능케
2 분야에서
2 데이터가
2 위해서는
2 필요한
2 기업
2 품질을
2 ETL
2 솔루션을
2 평가받고
2 비즈니스
2 등을
2 인터넷으로
2 이용해
2 예측할
2 모바일
2 IT
2 양의
2 구글은
2 수억
2 문장과
2 활용하여
2 자동번역
2 볼
1 Big
1 data
1 활용사례
1 의의[편집]
1 정치
1 사

### 3. 아래 데이터를 RDD로 만들고, 성적의 합계 및 평균을 계산하세요.

문항별로 RDD를 만들지 말고, 하나의 RDD만을 생성하고, 이를 변형해서 문제를 푸세요.



이름 | 과목 | 점수

-----|-----|-----

김하나 | English | 100

김하나 | Math | 80

임하나 | English | 70

임하나 | Math | 100

김갑돌 | English | 82.3

김갑돌 | Math | 98.5



### * 문제 3-1: 이름으로 합계를 구해보자.

collectByname

In [89]:
data = [("김하나","English",100), 
        ("김하나","Math",80), 
        ("임하나","English",70),
        ("임하나","Math",100),
        ("김갑돌","English",82.3),
        ("김갑돌","Math",98.5)
       ]

rdd = spark.sparkContext.parallelize(data)
byname = rdd.map(lambda x : (x[0],float(x[2])))
cbyname = byname.combineByKey(lambda value: (value, 1 ),
                             lambda x, value: (x[0]+value, x[1]+1),
                            lambda x,y: (x[0] + y[0] , x[1] + y[1]))

b = cbyname.map(lambda x :( x[0],x[1][0])).sortByKey(False).collect()

for i in b:
    print(i[0], i[1])

임하나 170.0
김하나 180.0
김갑돌 180.8


### * 문제 3-2: 과목으로 합계를 구해보자.

In [71]:
bysubject = rdd.map(lambda x: (x[1], x[2]))
cbysubject = bysubject.combineByKey(lambda value: (value, 1 ),
                             lambda x, value: (x[0]+value, x[1]+1),
                            lambda x,y: (x[0] + y[0] , x[1] + y[1]))
d = cbysubject.map(lambda x : (x[0], x[1][0])).collect()
for i in d:
    print(i[0], i[1])

English 252.3
Math 278.5


### * 문제 3-3: 이름으로 합계과 개수를 구해보자

In [69]:
a = cbyname.sortByKey(False).collect()
for i in a:
    print(i[0], i[1])

임하나 (170.0, 2)
김하나 (180.0, 2)
김갑돌 (180.8, 2)


In [92]:
b = cbyname.map(lambda x : (x[0], x[1][0]/x[1][1])).sortByKey(False).collect()
for k,v in b:
    print(k, v)

임하나 85.0
김하나 90.0
김갑돌 90.4


In [93]:
data = ["'김하나', 'English', 100",
        "'김하나', 'Math', 80",
        "'임하나', 'English', 70",
        "'임하나', 'Math', 100",
        "'김갑돌', 'English', 82.3",
        "'김갑돌', 'Math', 98.5"]

In [97]:
rdd2= spark.sparkContext.parallelize(data);
rdd2.map(lambda x : x.split()).map(lambda x : x[0],x[])

[["'김하나',", "'English',", '100'],
 ["'김하나',", "'Math',", '80'],
 ["'임하나',", "'English',", '70'],
 ["'임하나',", "'Math',", '100'],
 ["'김갑돌',", "'English',", '82.3'],
 ["'김갑돌',", "'Math',", '98.5']]